## 1. Introduction
Imagine that you're an IT specialist working in a medium-sized company, your manager wants to create a daily report that tracks the use of machines. Specifically, she wants to know which users are currently connected to which machines, it's your job to create the report. In your company, there's a system that collects every event that happens on the machines on the network. Among the many events collected it records each time a user logs in or out of a computer. With this information, we want to write a script that generates a report of which users are logged in to which machines at that time. 

Before we jump into solving that problem, we need to know what information we'll use as input and what information we'll have as output.

In our report scenario, the input is a _list of events_ , each event is an instance of the event class.

## 2. Event class
The __event__ class contains the _date_ when the event happened, the _name_ of the machine where it happened, the _user_ involved, and the event _type_

The attributes are called 
- `date` 
- `user` 
- `machine` 
- `type`

The event types are strings and the ones we care about are _login_ and _logout_

Our script will receive a list of event objects and we'll access the events attributes

We want to generate a report that lists all the machine names and for each machine, lists of the users that are currently logged in. We then want this information printed on the screen.

### Example

machine: _____

user: _____

date: _____

type: _____

We've been tasked with generating a report and we can decide exactly how we want that report to look. One option would be to print the name of the machine at the beginning of the line and then list the current users on separate lines and indent it to the right, or we could print the machine name followed by a colon and then the usernames separated by commas all in the same line, and we can probably come up with something even more fancy

Let's keep it simple for now and we'll go with the approach of printing the machine name followed by all the current users separated by commas

## 3. Research

We have our problem statement which helps us understand the problem and focus our approach. We know we have to input a list of event objects and evaluate these objects attributes to output a report of all the users currently logged into a machine. Now it's time for step 2, the research. We're going to consider all the tools we have available to help us solve the problem. To find out which users are currently logged into machines, we need to check when they logged in and when they logged out.

So how do we sort lists in Python? We'll need to do some research. Type sort lists in Python into your favorite search engine and you'll get a bunch of results that mentioned the list `sort` method and the `sorted` function. The difference between these two options is that the `sort` method modifies the list it's executed on, while the `sorted` function returns a new list that's been sorted. 

In [6]:
numbers = [2, 38, 1, 53, 46]
numbers.sort()
print(numbers)

[1, 2, 38, 46, 53]


In [8]:
names = ['Brian', 'Chester', 'AJ', 'Chad']
print(sorted(names))
print(names)

['AJ', 'Brian', 'Chad', 'Chester']
['Brian', 'Chester', 'AJ', 'Chad']


We now know how to sort things in Python. For this problem, it's fine to modify the original list. So we'll use the sort method. But wait, see how both these options sorted the list alphabetically? That's the default approach Python takes. But what if we wanted to organize our lists by different criteria? Again, if we take a look at the documentation we found online, we'll see that the sort method can take a couple of parameters. One of these parameters is called key, and it lets us use a function as the sorting key. Let's try this out on our list of names. Instead of sorting alphabetically, we could sort by the length of each string. Do you remember which function we can use to do that? Yes, we can pass the len function as the key.

In [9]:
print(sorted(names, key=len))

['AJ', 'Chad', 'Brian', 'Chester']


In our report scenario, we know that our elements will be instances of the event class and we want to order by date, which is an attribute of the event class. One way we could do this is to write a function called `get_event_date` which returns the date stored in the event object

## 4. Planning

So we know that our input will be a __list of events and we'll sort them by time.__ Each event in that list will include a machine name, a username, and tell us whether the event is a login or a logout. We want our script to keep track of users as they log in and out of machines. So how can we do this?

When we process an event, we'll see that someone interacted with a machine. If it's a login, we want to add it to the group of users logged into that machine. If it's a logout, we want to remove it from the group of users logged into the machine.

```python
# John has logged in
machine_users = ['John']

# John has logged out
machine_users = []
```

In this scenario, it makes sense to use a set to store the current users. Adding new users at login time and removing them at logout time. Great. But if the current users of a given machine are stored in a set, how do we know which set corresponds to the machine we're looking for? The easiest way to know this is to store this information in a dictionary.

We'll use the name of the machine as the key and the current users of the machine as the values. So for each event we process, we'll first check in the dictionary to see if the machine is already there. We need to check this because it could be the first time we're processing an event for that machine. If it's not there, we'll create a new entry. If it is, we'll update the existing entry with the action corresponding to the event. Which means we either add the user if the event is a login or remove the user if it's a logout. Once we're done processing the events, we'll want to print a report of the information we generated. This is a completely separate task. 

 It's important to have separate functions; to process the data and to print the data to the screen. This is because if we want to modify how the report is printed, we know we only need to change the function in charge of printing. Or, if we find a bug in our processing the data, we only need to change the processing function
 
 ## 5. The Script

In [10]:
def get_event_date(event):
    return event.date

def current_users(events):
    events.sort(key=get_event_date)
    machines = {}
    for event in events:
        if event.machine not in machines:
            machines[event.machine] = set()
        if event.type == 'login':
            machines[event.machine].add(event.user)
        elif event.type == 'logout':
            machines[event.machine].remove(event.user)
    return machines

def generate_report(machines):
    for machine, users in machines.items():
        # Print when the set of users have more than 0 elements
        if len(users) > 0:
            user_list = ', '.join(users)
            print('{}: {}'.format(machine, user_list))

In [16]:
class Event:
    def __init__(self, event_date, event_type, machine_name, user):
        self.date = event_date
        self.type = event_type
        self.machine = machine_name
        self.user = user

In [37]:
events = [
    Event('2020-01-21 12:45:34', 'login', 'myworkstation.local', 'brian'),
    Event('2020-01-30 01:45:34', 'login', 'webserver.local', 'chad'),
    Event('2020-01-19 12:45:09', 'login', 'myworkstation.local', 'brian'),
    Event('2020-01-3 13:45:47', 'login', 'webserver.local', 'brian'),
    Event('2020-01-1 17:00:00', 'login', 'myworkstation.local', 'kane')
]

In [38]:
users = current_users(events)
print(users)

{'myworkstation.local': {'kane', 'brian'}, 'webserver.local': {'chad', 'brian'}}


In [39]:
generate_report(users)

myworkstation.local: kane, brian
webserver.local: chad, brian
